In [ ]:
import pandas as pd
import numpy as np
import stockstats

In [ ]:
df = pd.read_csv('test.csv')
df

In [ ]:
# dataset['PeriodEnd'] = pd.to_datetime(dataset['PeriodEnd'])
# dataset.set_index('PeriodEnd', drop = False, inplace = True)
# Mt_Gambier= timezone('Australia/Adelaide')
# dataset.index = dataset.index.tz_localize('UTC').tz_convert(Mt_Gambier)
# dataset.head()
import pytz
from pytz import timezone

df['date'] = pd.to_datetime(df.iloc[:,0])
df = df.set_index('date')
df = df.iloc[:,1:]
df

In [ ]:
common_factors = pd.concat([df.mean(axis=1), df.median(axis=1)], axis=1)
common_factors.columns = ['mean', 'median']
factor_stats = stockstats.StockDataFrame.retype(common_factors)
common_factors['mean_60_ema'] =  factor_stats['mean_60_ema']
common_factors['median_60_ema'] =  factor_stats['median_60_ema']
common_factors

In [ ]:
import random
prices_df = pd.DataFrame()
for col in df.columns:
    base_price = random.randint(10,100)
    accumulate_return = (df[col].cumsum())+1
    accumulate_return = accumulate_return * base_price
    prices_df = pd.concat([prices_df, accumulate_return], axis=1)
prices_df.index.set_names(['date'], inplace=True)
prices_df.index = pd.to_datetime(prices_df.index)
prices_df = prices_df.shift(-1).dropna()
prices_df

In [ ]:
multiIndexFactors = pd.DataFrame()

returns_df = df.transpose()
returns_df.index.set_names(['code'], inplace=True)

for col in returns_df.columns:
    # 因子值
    factor_mean = (returns_df[col]-common_factors.loc[col,'mean'])/common_factors.loc[col,'mean_60_ema']
    factor_median = (returns_df[col]-common_factors.loc[col,'median'])/common_factors.loc[col,'median_60_ema']
    factor_stock = pd.concat([factor_mean, factor_median], axis=1)
    factor_stock.columns = ['mean_diff', 'median_diff']
    factor_stock['date']= col
    factor_stock.reset_index(inplace=True)
    multiIndexFactors = multiIndexFactors.append(factor_stock)

multiIndexFactors = multiIndexFactors.set_index(['date','code'])
from scipy.stats import zscoremultiIndexFactors = multiIndexFactors.apply(zscore)multiIndexFactors  

In [ ]:
import alphalens

factor_results = {}
for col in multiIndexFactors.columns:
    data = alphalens.utils.get_clean_factor_and_forward_returns(multiIndexFactors[col], prices_df, quantiles=5, periods=(1,5,20))
    factor_results[col] = data

In [ ]:
factor_results['mean_diff']

In [ ]:
alphalens.tears.create_full_tear_sheet(factor_results['median_diff'])